This script allow to get all the papers in openAlex that are including one or more keywords (in a list) in their title or abstract.
The keywords are considered here by ngrams.

In [3]:
import requests
import pandas as pd
from tqdm import tqdm

# open keywords database (AI)

In [7]:
AIkwclass = pd.read_csv('/../classification.txt',sep=';',names=['keyword','class'])
AIkwclass

,keyword,class
0,evolutionary algorithm,optimization
1,simulated annealing,optimization
2,hierarchical clustering,cluster analysis
3,principal component analysis,cluster analysis
4,unsupervised learning,machine learning
...,...,...
527,multi swarm optimization,optimization
528,fast and frugal trees,AI general
529,chess playing computer,robotics
530,bias variance tradeoff,optimization


# scrape papers for one keyword/ngram

In [26]:
def add_request_abstract(string, ngram):
    if string[-1] != '=':
        return string + ',abstract.search:' + ngram.replace(' ','%')
    else:
        return string + 'abstract.search:' + ngram.replace(' ','%')
    
def add_request_title(string, ngram):
    if string[-1] != '=':
        return string + ',title.search:' + ngram.replace(' ','%')
    else:
        return string + 'title.search:' + ngram.replace(' ','%')
    
def add_request_fulltext(string, ngram):
    if string[-1] != '=':
        return string + ',fulltext.search:' + ngram.replace(' ','%')
    else:
        return string + 'fulltext.search:' + ngram.replace(' ','%')
    
def add_publicationYear(string, year):
    if string[-1] != '=':
        return string + f',publication_year:{year}'
    else:
        return string + f'publication_year:{year}'
    
def rebuild_abstract(word_dict): # from the key 'abstract_inverted_index' key per paper
    last_word = list(word_dict.keys())[-1]
    abstract = [0]*(word_dict[last_word][-1]+1)
    for i in word_dict.keys():
        for j in word_dict[i]:
            abstract[j] = i
    abstract = ' '.join(abstract)
    return abstract

In [51]:
is_work = 'https://api.openalex.org/works?'
authentification = 'mailto=myemail@adress.com'
# need to have a doi, an abstract and at least one reference
filtering = 'filter=has_doi:true,has_abstract:true,has_references:true'
results_per_page = 'per-page=100'
cursor = 'cursor=*'

In [52]:
baseURL = '&'.join([is_work,
                    authentification,
                    results_per_page,
                    cursor,
                    filtering])
baseURL = add_request_abstract(baseURL, 'chess playing computer')
baseURL = add_request_title(baseURL, 'chess playing computer')
baseURL = add_publicationYear(baseURL, 2020)

In [46]:
baseURL

'https://api.openalex.org/works?&mailto=sylvain.fontaine@cnrs.fr&per-page=100&cursor=*&filter=has_doi:true,has_abstract:true,has_reference:true,abstract.search:chess%playing%computer,title.search:chess%playing%computer,publication_year:2020'

In [53]:
#request for one keyword
cc = requests.get(baseURL).json()

In [55]:
pd.DataFrame(cc['results'])

,id,doi,title,display_name,relevance_score,publication_year,publication_date,ids,host_venue,type,...,mesh,alternate_host_venues,referenced_works,related_works,ngrams_url,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date
0,https://openalex.org/W3047111047,https://doi.org/10.3233/icg-200148,An analysis of games played in world computer ...,An analysis of games played in world computer ...,4.0,2020,2020-01-01,{'openalex': 'https://openalex.org/W3047111047...,"{'id': 'https://openalex.org/V30686695', 'issn...",journal-article,...,[],[],"[https://openalex.org/W1995987707, https://ope...","[https://openalex.org/W880416, https://openale...",https://api.openalex.org/works/W3047111047/ngrams,"{'It': [0], 'is': [1], 'been': [2, 22], '46': ...",https://api.openalex.org/works?filter=cites:W3...,[],2022-08-02T01:15:27.794903,2020-08-10


# scrape papers for mutiple keywords/ngrams

In [51]:
is_work = 'https://api.openalex.org/works?'
authentification = 'mailto=myemail@adress.com'
# need to have a doi, an abstract and at least one reference
filtering = 'filter=has_doi:true,has_abstract:true,has_references:true'
results_per_page = 'per-page=100'

In [68]:
kw_list = [i[1:] for i in AIkwclass['keyword']]

In [81]:
papers = []
for kw in tqdm(kw_list):
    cursor = '*'
    l = 1
    while (cursor != None) & (l != 0):
        baseURL = '&'.join([is_work,
                        authentification,
                        results_per_page,
                        f'cursor={cursor}',
                        filtering])
        baseURL = add_request_abstract(baseURL, kw)
        baseURL = add_request_title(baseURL, kw)
        #baseURL = add_request_fulltext(baseURL, kw)
        response = requests.get(baseURL).json()
        papers.extend(response['results'])
        cursor = response['meta']['next_cursor']
        l = len(response['results'])
    # break # for one keyword testing

  0%|          | 0/532 [01:36<?, ?it/s]
